<a href="https://colab.research.google.com/github/blackbudge98-cpu/gt-markets/blob/blackbudge98-cpu-patch-1/Preprocess_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Keywords as a predictive indicator of USD trading performance

The Project is exploring how Google Trend KeyWord Data can be used in forward validation to determine the probability of a movement in a trading pair.

Our Control Variable will be USD to determine's its performance on the following trading pairs:

USD to Chinese Yuan

*   USD to BTC
* USD to Oil
* USD to Gold




In [52]:
#Use of the following libaries will assist in providing the project manager with the data

import yfinance as yf
import pandas as pd
from datetime import date

#The first data set we will want to see is USD over a 10 year period

tickers = ["USD", "USDCNY=X","BTC-USD","CL=F", "GC=F"]

#The definition of tickers will assist in a batch query rather than a singular batch query

df = yf.download(tickers, period="10y", interval="1d")["Close"]

#Rename the columns to be more user friendly, and align with our assumptions

df.rename(columns={"CL=F":"USD to Oil","GC=F":"USD to Gold","BTC-USD": "USD to BTC","USDCNY=X": "USD to Chinese Yuan"},inplace=True)

#print headers for 10 rows to see what the data looks like

print(df.head(10))

/tmp/ipython-input-924416535.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, period="10y", interval="1d")["Close"]
[*********************100%***********************]  5 of 5 completed

Ticker      USD to BTC  USD to Oil  USD to Gold       USD  USD to Chinese Yuan
Date                                                                          
2015-08-30  228.761002         NaN          NaN       NaN                  NaN
2015-08-31  230.056000   49.200001  1131.599976  1.390877               6.3785
2015-09-01  228.121002   45.410000  1138.699951  1.281002               6.3664
2015-09-02  229.283997   46.250000  1132.500000  1.341353               6.3545
2015-09-03  227.182999   46.750000  1123.699951  1.383659               6.3459
2015-09-04  230.298004   46.049999  1120.599976  1.345163               6.3459
2015-09-05  235.018997         NaN          NaN       NaN                  NaN
2015-09-06  239.839996         NaN          NaN       NaN                  NaN
2015-09-07  239.847000         NaN          NaN       NaN               6.3459
2015-09-08  243.606995   45.939999  1120.400024  1.448019               6.3559


In [53]:
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
from datetime import date

# Authenticate and mount Google Drive
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Define the folder ID and filename
folder_id = '1tqNeIkQM2IawFLS-NHBzaDl8fsJAo0t_'
today = date.today()
filename = f"financial_data_raw_data_from_yf{today.strftime('%Y-%m-%d')}.csv"
filepath = f"/content/{filename}" # Save locally first

# Save the DataFrame to a temporary local CSV file
df.to_csv(filepath)

# Create a file in the shared drive
file_metadata = {
    'name': filename,
    'parents': [folder_id]
}

media = MediaFileUpload(filepath, mimetype='text/csv')

gfile = drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()

print(f"File ID: {gfile.get('id')}")
print(f"Data exported to shared drive folder: {folder_id}")

# Clean up the local file
os.remove(filepath)

Mounted at /content/drive
File ID: 1cbd4Z2Kcegp1jtWc8dWfdPh73livcoEH
Data exported to shared drive folder: 1tqNeIkQM2IawFLS-NHBzaDl8fsJAo0t_


Install of pytrends to get a daily view of key words that are applicable. Chose the price of each trading pair and the federal reserve.

In [54]:
!pip install pytrends

from pytrends.request import TrendReq
from datetime import date, timedelta

# Initialize TrendReq
pytrends = TrendReq(hl='en-US', tz=360)

# Define the keyword and the specific time periods
# Using more specific keywords related to USD and Gold that might have more search interest
kw_list = ["USD price", "Gold price", "Chinese Yuan Price", "Oil Price", "Federal Reserve"]

# Define the dates for the timeframe (you can adjust these as needed)
date_max_change = date.today()
# Change timedelta to get the last 3 months for daily data
date_min_change = date_max_change - timedelta(days=90) # Approximately 3 months

# Format the dates for pytrends
date_max_change_str = date_max_change.strftime('%Y-%m-%d')
date_min_change_str = date_min_change.strftime('%Y-%m-%d')

# Define the timeframes for the specific dates
# Combine the date range into a single timeframe
timeframe_combined = f'{date_min_change_str} {date_max_change_str}'

# Set geo to 'US' for United States and try a finance category (cat=7 for Finance)
print(f"Google Trends data for 'USD price' and 'Gold price' in the United States between {date_min_change_str} and {date_max_change_str}:")
try:
    pytrends.build_payload(kw_list, cat=7, timeframe=timeframe_combined, geo='US', gprop='')
    pytrends_usd_dataframe = pytrends.interest_over_time()
    display(pytrends_usd_dataframe)
except Exception as e:
    print(f"Could not retrieve data for the date range or parameters: {e}")

Google Trends data for 'USD price' and 'Gold price' in the United States between 2025-06-01 and 2025-08-30:


/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


,USD price,Gold price,Chinese Yuan Price,Oil Price,Donald Trump,isPartial
date,,,,,,
2025-06-01,4,42,0,3,1,False
2025-06-02,6,78,0,5,1,False
2025-06-03,5,76,0,5,1,False
2025-06-04,5,71,0,5,1,False
2025-06-05,6,76,0,4,2,False
...,...,...,...,...,...,...
2025-08-26,6,61,0,5,1,False
2025-08-27,6,59,0,4,1,False
2025-08-28,5,63,0,4,1,False


In [55]:
# Filter the financial data to the last 90 days to match the Google Trends data timeframe
from datetime import date, timedelta

date_max_change = date.today()
date_min_change = date_max_change - timedelta(days=90)

df_last_90_days = df.loc[date_min_change:date_max_change]

# Merge the financial data (last 90 days) with the Google Trends data (first set of keywords)
# We use an outer merge to keep all dates from both dataframes, filling missing values with NaN
merged_df = pd.merge(df_last_90_days, pytrends_usd_dataframe, left_index=True, right_index=True, how='outer')

# Merge the combined DataFrame with the second set of Google Trends data
merged_df = pd.merge(merged_df, pytrends_inflation_dataframe, left_index=True, right_index=True, how='outer')

# Merge the combined DataFrame with the third set of Google Trends data
merged_df = pd.merge(merged_df, pytrends_third_dataframe, left_index=True, right_index=True, how='outer')

# Display the merged DataFrame
display(merged_df.head())
display(merged_df.tail())

,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,Donald Trump,...,intervention,gamble,luxury,isPartial_y,blackmail,privilege,ghetto,extravagant,nobility,isPartial
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-01,105652.101562,NaN,NaN,NaN,NaN,4,42,0,3,1,...,0,0,13,False,0,0,0,0,0,False
2025-06-02,105881.531250,62.520000,3370.600098,56.934940,7.2037,6,78,0,5,1,...,0,0,15,False,0,0,0,0,0,False
2025-06-03,105432.468750,63.410000,3350.199951,60.107414,7.2037,5,76,0,5,1,...,0,0,14,False,0,0,0,0,0,False
2025-06-04,104731.984375,62.849998,3373.500000,60.985332,7.2037,5,71,0,5,1,...,0,0,16,False,0,0,0,0,0,False
2025-06-05,101575.953125,63.369999,3350.699951,60.057533,7.2037,6,76,0,4,2,...,0,0,21,False,0,0,0,0,0,False


,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,Donald Trump,...,intervention,gamble,luxury,isPartial_y,blackmail,privilege,ghetto,extravagant,nobility,isPartial
Date,,,,,,,,,,,,,,,,,,,,,
2025-08-26,111802.656250,63.250000,3388.600098,89.540001,7.1510,6,61,0,5,1,...,1,4,13,False,0,31,7,0,0,False
2025-08-27,111222.062500,64.150002,3404.600098,89.720001,7.1520,6,59,0,4,1,...,2,5,13,False,0,27,10,4,0,False
2025-08-28,112544.804688,64.599998,3431.800049,89.980003,7.1530,5,63,0,4,1,...,1,6,12,False,0,35,6,0,0,False
2025-08-29,108410.835938,64.010002,3473.699951,84.370003,7.1299,6,74,0,4,1,...,1,4,12,False,0,24,10,0,0,False
2025-08-30,108693.843750,NaN,NaN,NaN,NaN,5,58,0,3,2,...,1,4,17,True,10,32,10,0,0,True


In [56]:
from pytrends.request import TrendReq
from datetime import date, timedelta

# Initialize TrendReq
pytrends = TrendReq(hl='en-US', tz=360)

# Define the second list of keywords
kw_list_2 = ["inflation", "intervention", "gamble", "luxury", "ruin"]

# Define the dates for the timeframe (last 3 months for daily data)
date_max_change_2 = date.today()
date_min_change_2 = date_max_change_2 - timedelta(days=90)

# Format the dates for pytrends
date_max_change_str_2 = date_max_change_2.strftime('%Y-%m-%d')
date_min_change_str_2 = date_min_change_2.strftime('%Y-%m-%d')

# Define the timeframe for the specific dates
timeframe_combined_2 = f'{date_min_change_str_2} {date_max_change_str_2}'

# Set geo to 'US' for United States and try a finance category (cat=7 for Finance)
print(f"Google Trends data for {kw_list_2} in the United States between {date_min_change_str_2} and {date_max_change_str_2}:")
try:
    pytrends.build_payload(kw_list_2, cat=7, timeframe=timeframe_combined_2, geo='US', gprop='')
    pytrends_inflation_dataframe = pytrends.interest_over_time()
    display(pytrends_inflation_dataframe)
except Exception as e:
    print(f"Could not retrieve data for the date range or parameters: {e}")

Google Trends data for ['inflation', 'intervention', 'gamble', 'luxury', 'ruin'] in the United States between 2025-06-01 and 2025-08-30:


/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


,inflation,intervention,gamble,luxury,ruin,isPartial
date,,,,,,
2025-06-01,32,0,0,10,0,False
2025-06-02,41,0,0,13,0,False
2025-06-03,41,0,0,13,0,False
2025-06-04,36,0,5,15,0,False
2025-06-05,51,0,0,18,0,False
...,...,...,...,...,...,...
2025-08-26,54,1,4,14,2,False
2025-08-27,57,2,5,13,2,False
2025-08-28,56,1,6,13,3,False


In [57]:
from pytrends.request import TrendReq
from datetime import date, timedelta

# Initialize TrendReq
pytrends = TrendReq(hl='en-US', tz=360)

# Define the third list of keywords
kw_list_3 = ["blackmail", "privilege", "ghetto", "extravagant", "nobility"]

# Define the dates for the timeframe (last 3 months for daily data)
date_max_change_3 = date.today()
date_min_change_3 = date_max_change_3 - timedelta(days=90)

# Format the dates for pytrends
date_max_change_str_3 = date_max_change_3.strftime('%Y-%m-%d')
date_min_change_str_3 = date_min_change_3.strftime('%Y-%m-%d')

# Define the timeframe for the specific dates
timeframe_combined_3 = f'{date_min_change_str_3} {date_max_change_str_3}'

# Set geo to 'US' for United States and try a finance category (cat=7 for Finance)
print(f"Google Trends data for {kw_list_3} in the United States between {date_min_change_str_3} and {date_min_change_str_3}:")
try:
    pytrends.build_payload(kw_list_3, cat=7, timeframe=timeframe_combined_3, geo='US', gprop='')
    pytrends_third_dataframe = pytrends.interest_over_time()
    display(pytrends_third_dataframe)
except Exception as e:
    print(f"Could not retrieve data for the date range or parameters: {e}")

Google Trends data for ['blackmail', 'privilege', 'ghetto', 'extravagant', 'nobility'] in the United States between 2025-06-01 and 2025-06-01:


/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


,blackmail,privilege,ghetto,extravagant,nobility,isPartial
date,,,,,,
2025-06-01,0,0,0,0,0,False
2025-06-02,0,0,0,0,0,False
2025-06-03,0,0,0,0,0,False
2025-06-04,0,0,0,0,0,False
2025-06-05,0,0,0,0,0,False
...,...,...,...,...,...,...
2025-08-26,0,31,7,0,0,False
2025-08-27,0,27,10,4,0,False
2025-08-28,0,35,6,0,0,False


In [58]:
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
from datetime import date, timedelta

# Authenticate and mount Google Drive (assuming it's not already mounted and authenticated)
# If you have already run the cell S_kM5CWcaq_D, you can comment out these lines
# drive.mount('/content/drive', force_remount=True)
# auth.authenticate_user()
# drive_service = build('drive', 'v3')

# Define the folder ID
folder_id = '1tqNeIkQM2IawFLS-NHBzaDl8fsJAo0t_' # Replace with your actual folder ID

# Define the filename based on the date range
today = date.today()
ninety_days_ago = today - timedelta(days=90)
filename = f"{today.strftime('%Y-%m-%d')}_{ninety_days_ago.strftime('%Y-%m-%d')}_Merged_raw_data_base.csv"
filepath = f"/content/{filename}" # Save locally first

# Save the merged DataFrame to a temporary local CSV file
merged_df.to_csv(filepath)

# Create a file in the shared drive
file_metadata = {
    'name': filename,
    'parents': [folder_id]
}

media = MediaFileUpload(filepath, mimetype='text/csv')

# Use the existing drive_service if available, otherwise create a new one
try:
    gfile = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()
    print(f"File ID: {gfile.get('id')}")
    print(f"Data exported to shared drive folder: {folder_id}")
except NameError:
    print("drive_service not found. Please ensure the Google Drive authentication cell has been run.")


# Clean up the local file
os.remove(filepath)

File ID: 1xv-RWPhVqGSMolO5fhEfdbbb350c-bIT
Data exported to shared drive folder: 1tqNeIkQM2IawFLS-NHBzaDl8fsJAo0t_
